<a href="https://colab.research.google.com/github/lucasmbarboza/ICUFF_ML/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy 
!pip install pandas
!pip install tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!cd /content/drive/MyDrive/IA/dataset/ 

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
filepath = '/content/drive/MyDrive/IA/dataset/DIAGNOSES_ICD.csv'
dataset_dig9 = pd.read_csv(filepath, sep=',', decimal='.')

In [ ]:
def get_unique( data , columm ):
  uniq_list = data[columm].unique().tolist()
  return uniq_list

In [ ]:
diagnoses = get_unique(data=dataset_dig9 , columm='icd9_code')
patients  = get_unique(data=dataset_dig9 , columm='subject_id')
admitions = get_unique(data=dataset_dig9 , columm='hadm_id')
print(diagnoses)

['99591', '99662', '5672', '40391', '42731', '4280', '4241', '4240', '2874', '03819', '7850', 'E8791', 'V090', '56211', '28529', '25000', 'V5867', 'E9342', '41401', '2749', '3051', '570', '07030', '07054', '30401', '2875', '2760', '0389', '41071', '78551', '486', '20280', '4582', '2724', '81201', '4928', '8028', '8024', '99812', '41511', '2851', 'E8859', 'E8788', 'V1259', '4019', '51881', '5770', '30390', '5781', '5845', '2848', '5722', '78559', '99592', '5711', '49390', '7299', '431', '4010', '5990', '2761', 'V103', '3569', '2859', '3970', '4266', '5859', '2948', '5691', '79092', '0380', '99673', '5761', '57451', '5856', '4538', '78552', '51882', '5119', '99659', '7211', 'V1588', '2380', '2930', '0414', 'V4502', 'V1046', '81249', '1985', '5070', '42830', '2765', '5849', '24220', '51919', '2767', 'E8889', '2720', '311', '43811', '53390', '43310', '6000', '412', '2762', '2554', '53081', '3310', '29410', '725', '2449', '99667', '71105', '2763', '99677', '7837', '45981', 'E8781', 'E8498',

In [ ]:
len(patients)

100

In [ ]:
'''
Pacienteid: { 
   admitions: []
   diagnoses: [] 
}
'''
Mapdict = {}
for patient in patients: 
  Mapdict[str(patient)] = {'admitions':dataset_dig9.loc[dataset_dig9['subject_id'] == patient]['hadm_id'].unique().tolist() , 'diagnoses':dataset_dig9.loc[dataset_dig9['subject_id'] == patient]['icd9_code'].unique().tolist()}

In [ ]:
diagnoses.sort()

In [ ]:
''' 
 \  D0  D1 ... Dn 
 P0 1   0  ... 0
 P1 0   0  ... 1
 ... 
 Pn 1   1  ... 1
 '''
Bigvet = []
for patient in patients: 
  patVet = [] 
  for diagnostic in diagnoses:
    if diagnostic in Mapdict[str(patient)]['diagnoses']: 
      patVet.append(1)
    else: 
      patVet.append(0)
  Bigvet.append(patVet) 

In [ ]:
Nw = pd.DataFrame(Bigvet)
Nw.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580
count,100.000000,100.000000,100.000000,100.000000,100.00,100.000000,100.00,100.00,100.00,100.000000,100.00,100.000000,100.00,100.00,100.00,100.000000,100.00,100.00000,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.000000,100.000000,100.00,100.00,100.00,100.000000,100.00,100.00,100.00,100.00,100.00,100.00,100.00,...,100.00,100.000000,100.000000,100.000000,100.000000,100.00,100.00,100.00,100.00,100.00,100.00,100.000000,100.00,100.00,100.00,100.000000,100.00,100.00,100.00,100.00,100.00,100.000000,100.000000,100.000000,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.000000,100.00,100.000000,100.00,100.000000,100.000000,100.000000
mean,0.070000,0.020000,0.020000,0.020000,0.01,0.020000,0.01,0.01,0.01,0.020000,0.01,0.160000,0.01,0.01,0.01,0.020000,0.01,0.08000,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.020000,0.020000,0.01,0.01,0.01,0.030000,0.01,0.01,0.01,0.01,0.01,0.01,0.01,...,0.01,0.030000,0.040000,0.020000,0.050000,0.01,0.01,0.01,0.01,0.01,0.01,0.020000,0.01,0.01,0.01,0.020000,0.01,0.01,0.01,0.01,0.01,0.070000,0.040000,0.020000,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.090000,0.01,0.050000,0.01,0.030000,0.030000,0.020000
std,0.256432,0.140705,0.140705,0.140705,0.10,0.140705,0.10,0.10,0.10,0.140705,0.10,0.368453,0.10,0.10,0.10,0.140705,0.10,0.27266,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.140705,0.140705,0.10,0.10,0.10,0.171447,0.10,0.10,0.10,0.10,0.10,0.10,0.10,...,0.10,0.171447,0.196946,0.140705,0.219043,0.10,0.10,0.10,0.10,0.10,0.10,0.140705,0.10,0.10,0.10,0.140705,0.10,0.10,0.10,0.10,0.10,0.256432,0.196946,0.140705,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.10,0.287623,0.10,0.219043,0.10,0.171447,0.171447,0.140705
min,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.0

In [ ]:
def noise(size): 
  return np.zeros(shape=(size,581))

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import time
import matplotlib.pyplot as plt

In [ ]:
''' 
x = [0 1 .. 1]
Dis(x) = [p1 ... pn]
Z = [1 1 .. 1] noise vector
gen(Z) = [1 0 .. 1]  
Dis(gen(z)) = [^p1 ^p2 ... ^pn]
'''

' \nx = [0 1 .. 1]\nDis(x) = [p1 ... pn]\nZ = [1 1 .. 1] noise vector\ngen(Z) = [1 0 .. 1]  \nDis(gen(z)) = [^p1 ^p2 ... ^pn]\n'

In [ ]:
def generator(hsize=[10, 10],reuse=False):
    # Cria o modelo / Funciona como stack de opercoes
    model =  tf.keras.Sequential()

    # Cria primeira camada da rede neural usando os input Z / layer.dense cria uma rede neural do tipo densa 
    model.add(tf.keras.layers.Dense(hsize[1],name='GL1' ,activation=tf.keras.layers.LeakyReLU(),input_shape=(BATCH_SIZE,581)))

    # Cria a segunda camada da rede usando o input h1(Z)
    model.add( tf.keras.layers.Dense(hsize[1],name='GL2',activation=tf.keras.layers.LeakyReLU()))

    # Cria a de saida camada da rede usando o input h2(h1(Z))
    model.add(tf.keras.layers.Dense(581,name='GLout', activation='relu')) 
    return model

In [ ]:
def discriminator(hsize=[10, 10],reuse=False):
    # Cria o modelo / Funciona como stack de opercoes
    model =  tf.keras.Sequential()
    # Cria primeira camada da rede neural usando os input Z / layer.dense cria uma rede neural do tipo densa 
    model.add(tf.keras.layers.Dense(hsize[0],name='DL1'  ,activation='sigmoid',input_shape=(BATCH_SIZE,581)))
    # Cria a segunda camada da rede usando o input h1(Z)
    model.add( tf.keras.layers.Dense(hsize[1], name='DL2',activation=tf.keras.layers.LeakyReLU()))
    model.add( tf.keras.layers.Dense(hsize[1],name='DL3', activation=tf.keras.layers.LeakyReLU()))
    # Cria a de saida camada da rede usando o input h2(h1(Z))
    model.add(tf.keras.layers.Dense(2, name='DL4'))
    # Cria a de saida camada da rede usando o input h3(h2(h1(Z)))
    model.add(tf.keras.layers.Dense(1,name='DLout', activation='relu'))
    return model

In [ ]:
EPOCHS = 10000
BATCH_SIZE = 10

In [ ]:
gen = generator()
dis = discriminator()

In [ ]:
Z = noise(10)
print(Z.shape)

(10, 581)


In [ ]:
fake_data = gen(Z, training=False)
print(fake_data)

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(10, 581), dtype=float32)


In [ ]:
decision = dis(fake_data, training=False)

In [ ]:
print(decision)

tf.Tensor(
[[0.13123296]
 [0.13123296]
 [0.13123296]
 [0.13123296]
 [0.13123296]
 [0.13123296]
 [0.13123296]
 [0.13123296]
 [0.13123296]
 [0.13123296]], shape=(10, 1), dtype=float32)


In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
# Otimizador
generator_optimizer = tf.keras.optimizers.Adam(1e-3,)
#generator_optimizer =tf.keras.optimizers.RMSprop(1e-3)

discriminator_optimizer = tf.keras.optimizers.Adam(1e-3)
#discriminator_optimizer = tf.keras.optimizers.RMSprop(1e-4)

In [ ]:
epoch_vec=[]
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
from random import randint

In [ ]:
len(Bigvet)

100

In [ ]:
def sample_data(matrix,batch):
  sample =[]
  for integer in range(batch): 
    sample.append(matrix[randint(0,len(matrix)-1)])
  #print(sample)
  return np.array(sample)

In [ ]:
@tf.function
def train_step(Y):
    Z = noise(BATCH_SIZE) #
    with tf.GradientTape(persistent=True) as tape:
      #print('Entrei no loop')
      fake_data = gen(Z, training=True)  
      real_output = dis(Y, training=True)
      fake_output = dis(fake_data, training=True)
      print(gen.)
      gen_loss = generator_loss(fake_output)
      #print(gen_loss)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = tape.gradient(gen_loss, gen.trainable_variables)
    gradients_of_discriminator = tape.gradient(disc_loss, dis.trainable_variables)
    del tape
    generator_optimizer.apply_gradients(zip(gradients_of_generator, gen.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, dis.trainable_variables))


In [ ]:
gen.summary()

Model: "sequential_35"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
GL1 (Dense)                  (None, 10, 10)            5820      
_________________________________________________________________
GL2 (Dense)                  (None, 10, 10)            110       
_________________________________________________________________
GLout (Dense)                (None, 10, 581)           6391      
Total params: 12,321
Trainable params: 12,321
Non-trainable params: 0
_________________________________________________________________


In [ ]:
dis.summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
DL1 (Dense)                  (None, 10, 10)            5820      
_________________________________________________________________
DL2 (Dense)                  (None, 10, 10)            110       
_________________________________________________________________
DL3 (Dense)                  (None, 10, 10)            110       
_________________________________________________________________
DL4 (Dense)                  (None, 10, 2)             22        
_________________________________________________________________
DLout (Dense)                (None, 10, 1)             3         
Total params: 6,065
Trainable params: 6,065
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epoch_vec = []
for x in range(EPOCHS):
    Y = sample_data(Bigvet,batch= BATCH_SIZE)
    train_step(Y)
    if x in [2500,5000,7500,10000]: 
      epoch_vec.append(gen(Z,training=False))

In [ ]:
z = noise(BATCH_SIZE)
fake = gen(Z, training=False)
print(fake)

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(10, 581), dtype=float32)


In [ ]:
print(dis(fake, training=False))

tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float32)
